In [ ]:
import numpy as np

#Função que retorna um dicionário contendo os vértices e suas respectivas arestas (lista de adjacências) de acordo com o grafo binário passado
def ler_grafo(grafo:str) -> dict:
  dict_grafo = {}
  with open(grafo,'r') as arquivo:
    linhas = arquivo.readlines()
    for i,linha in enumerate(linhas[1:], start=1):
      arestas = []
      if ' ' in linha:
        string = linha.replace(' ','')
      for j,col in enumerate(string.rstrip('\n'), start=1):
        if col == '1':
          arestas.append(j)
      dict_grafo[i] = arestas

  return dict_grafo

In [ ]:
#Variável recebendo as informações do grafo passado (vértices e suas respectivas arestas)
entrada = 'graph_2'
grafo = ler_grafo(f'entradas/{entrada}')
print(grafo)

{1: [3, 5, 7], 2: [6, 8], 3: [1, 4, 5, 6, 10], 4: [3, 5], 5: [1, 3, 4], 6: [2, 3], 7: [1, 8], 8: [2, 7, 9], 9: [8, 10], 10: [3, 9]}


In [ ]:
#Como a função de profundidade é recursiva utilizamos essas variáveis abaixo para armazenar os valores anteriores a pilha de chamadas
t = 0
qt_vertices = len(grafo)
PE = np.zeros(qt_vertices,dtype=int)
PS = np.zeros(qt_vertices,dtype=int)
pai = np.zeros(qt_vertices,dtype=int)
pais = []
irmaos = []
#Parâmetros: O grafo, o vértice inicial e um booleano pra que não fique criando os arquivos todas vez que eh executado
def busca_profundidade(grafo,inicial,escrita=False):
  global t
  #Contador para profundidade de entrada
  t = t + 1
  PE[inicial - 1] = t
  #Util apenas na fase de teste \/
  #print("Vizinhos de ",inicial," = ",grafo[inicial])
  for w in grafo[inicial]:
    if(PE[w - 1] == 0):
      PE[w - 1] = t
      pai[w - 1] = inicial
      aux = [inicial,w]
      aux.sort()
      pais.append(aux)
      #Executando a busca recursivamente sem gerar o arquivo
      busca_profundidade(grafo,w)
    else:
      #Condição de ser uma aresta de ser irmão
      if(PS[w - 1] == 0 and w != pai[inicial - 1]):
        aux = [inicial,w]
        aux.sort()
        irmaos.append(aux)
  #Contador para profundidade de saída baseado no último valor do contador de entrada
  t = t + 1
  #Quanto o vertice sair da pilha de execução, recebera t de saída
  PS[inicial - 1] = t
  if escrita:
    #Os sort são para facilitar na escrita, para que não tenha que verificar se [x,y] ou [y,x] in arestas.
    pais.sort()
    irmaos.sort()
    arestas = pais + irmaos
    arestas.sort()

    with open(f'{entrada}_dfs.gdf','w') as arquivo:
      arquivo.write("nodedef>name VARCHAR,label VARCHAR\n")
      for key in grafo.keys():
          arquivo.write(f'{key},{key}\n')
      arquivo.write('edgedef>node1 VARCHAR,node2 VARCHAR,directed BOOLEAN,color VARCHAR\n')
      for i in range(len(arestas)):
        if(arestas[i] in pais):
          arquivo.write(f"{arestas[i][0]},{arestas[i][1]},false,'0,0,255'\n")
        else:
          arquivo.write(f"{arestas[i][0]},{arestas[i][1]},false,'255,0,0'\n")

In [ ]:
#Parâmetros: O grafo, o vértice inicial e um booleano pra que não fique criando os arquivos todas vez que eh executado
def busca_largura(grafo,inicial,metrica=False):
  t = 0
  qt_vertices = len(grafo)
  PE = np.zeros(qt_vertices,dtype=int)
  Nivel_bfs = np.zeros(qt_vertices,dtype=int)
  pai = np.zeros(qt_vertices,dtype=int)
  pais = []
  irmaos = []
  primos = []
  tios = []
  t = t + 1
  PE[inicial - 1] = t
  Nivel_bfs[inicial - 1] = 0
  fila = []
  fila.append(inicial)

  while(len(fila) > 0):
    v = fila[0]
    fila.pop(0)

    for w in grafo[v]:
      if(PE[w - 1] == 0):
        t = t + 1
        PE[w - 1] = t
        Nivel_bfs[w - 1] = Nivel_bfs[v - 1] + 1
        fila.append(w)
        #print("O vertice ",v,"é PAI do vertice ",w)
        pai[w - 1] = v
        aux = [v,w]
        aux.sort()
        pais.append(aux)
      else:
        if(Nivel_bfs[w - 1] == Nivel_bfs[v - 1]):
          if(w in fila):
            if(pai[w - 1] == pai[v - 1]):
              #print("O vertice ",v,"é IRMAO do vertice ",w)
              aux = [v,w]
              aux.sort()
              irmaos.append(aux)
            else:
              #print("O vertice",v,"é PRIMO do vertice",w)
              aux = [v,w]
              aux.sort()
              primos.append(aux)
        elif(Nivel_bfs[w - 1] == Nivel_bfs[v - 1] + 1 and pai[w - 1] != pai[v - 1]):
          if(w in fila):
            #print("O vertice",v,"é TIO do vertice",w)
            aux = [v,w]
            aux.sort()
            tios.append(aux)

  #Os sort são para facilitar na verificação do arquivo e não ter que buscar a aresta ao contrário.
  pais.sort()
  irmaos.sort()
  primos.sort()
  tios.sort()
  arestas = pais + irmaos + primos + tios
  arestas.sort()

  if not metrica:
    with open(f'{entrada}_bfs.gdf','w') as arquivo:
      arquivo.write("nodedef>name VARCHAR,label VARCHAR\n")
      for key in grafo.keys():
          arquivo.write(f'{key},{key}\n')
      arquivo.write('edgedef>node1 VARCHAR,node2 VARCHAR,directed BOOLEAN,color VARCHAR\n')
      for i in range(len(arestas)):
        if(arestas[i] in pais):
          arquivo.write(f"{arestas[i][0]},{arestas[i][1]},false,'0,0,255'\n")
        elif(arestas[i] in irmaos):
          arquivo.write(f"{arestas[i][0]},{arestas[i][1]},false,'255,0,0'\n")
        elif(arestas[i] in primos):
          arquivo.write(f"{arestas[i][0]},{arestas[i][1]},false,'255,255,0'\n")
        elif(arestas[i] in tios):
          arquivo.write(f"{arestas[i][0]},{arestas[i][1]},false,'0,255,0'\n")

  return np.sum(Nivel_bfs),np.max(Nivel_bfs)

In [ ]:
#Definindo o vértice inicial para construção dos grafos
inicial = 1
#Gerando o arquivo contendo o grafo por busca em profundidade
busca_profundidade(grafo,inicial,True)

In [ ]:
#Calculando as métricas: raio, diâmetro e a distância média entre dois vértices
#Gerando o arquivo contendo o grafo por busca em largura
exc,diametro = busca_largura(grafo,inicial)
raio = diametro
soma_excentricidade = exc
n = len(grafo)
for i in range(1,n + 1):
  if i != inicial:
    #Executando a busca em largura por cada vértice sem gerar o arquivo
    nova_exc,novo_diametro = busca_largura(grafo,i,True)
    novo_raio = novo_diametro
    soma_excentricidade += nova_exc
    if novo_diametro > diametro:
      diametro = novo_diametro
    if novo_raio < raio:
      raio = novo_raio
# A distânia média entre dois vértices é calculado pelo somatório de todos os níveis da busca em largura (excentricidades), por vértice, dividido pelo arranjo do número de vértices (n) dois a dois
distancia_media = soma_excentricidade/(n*(n-1))

print('*'*50)
print(f'Diâmetro: {diametro}')
print(f'Raio: {raio}')
print(f'Comprimento médio do caminho: {distancia_media}')
print('*'*50)

**************************************************
Diâmetro: 4
Raio: 3
Comprimento médio do caminho: 2.0444444444444443
**************************************************
